<a href="https://colab.research.google.com/github/aaronmj7/PySpark-Assignment/blob/main/PySpark_Assignment_Sep23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADS2 - Assignment 1 - Data Handling and Processing with PySpark

**STUDENT NAME - Aaron Modiyil Joseph**

**STUDENT ID - 22018497**

In this assignment, you will be analysing the popularity of films and TV shows on the streaming platform, Netflix. Using your knowledge of PySpark DataFrames and Spark SQL, you will produce a number of "downstream" data products to analyse trends in global streaming habits.

Download the dataset from this [Kaggle](https://www.kaggle.com/dhruvildave/netflix-top-10-tv-shows-and-films) page. A copy of the `all_weeks_countries.csv` file is also available on the canvas page for this assignment.

Your task is to load in the data and produce a number of "downstream" data products and plots as described below.

The PySpark installation and setup is provided below for conveinience.

IMPORTANT: DO NOT EDIT OR REMOVE THE COMMENT TAGS IN THE CODE CELLS

In [32]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [33]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [35]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [36]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [37]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.
netflixcsvpath = '/content/drive/MyDrive/all-weeks-countries.csv'

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [52]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [53]:
# CodeGrade Tag Ex1a

### Display the table and its schema

netflixDF.show()
netflixDF.printSchema()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                null|                         1|
|   Argentina|          AR|2022-08

In [54]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF

filmsDF = netflixDF.filter('category == "Films"')

tvDF = netflixDF.filter('category == "TV"')

In [55]:
# CodeGrade Tag Ex1c

### Drop the 'season_title' column from the Films DataFrame, display the table

filmsDF = filmsDF.drop('season_title')
filmsDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

In [56]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.

tvDF = tvDF.withColumn('season_title',
                       F.when(F.isnull('season_title'), F.col('show_title'))
                       .otherwise(F.col('season_title')))
tvDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          2|        Another Self|Another Self: Sea...|                         4|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          4|            Manifest|  Manifest: Season 1|                         6|
|   Argentina|          AR|2022-08

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [57]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.

STDF = tvDF.where('country_name == "United Kingdom" and\
                   show_title == "Stranger Things"')

### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.

STWeeks = STDF.groupBy('show_title')\
              .agg(F.count_distinct('week').alias('weeks_in_top_10'))
STWeeks.show()

+---------------+---------------+
|     show_title|weeks_in_top_10|
+---------------+---------------+
|Stranger Things|             13|
+---------------+---------------+



# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [58]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25

Top25 = (tvDF.filter('country_name == "United Kingdom"')
        .groupby('season_title')
        .agg(F.max('cumulative_weeks_in_top_10').alias('weeks_in_top_10'))
        .sort(F.desc('weeks_in_top_10'))
        .limit(25))
Top25.show(25)

+--------------------+---------------+
|        season_title|weeks_in_top_10|
+--------------------+---------------+
|   Stranger Things 4|             13|
|     Ozark: Season 4|             12|
|   Stranger Things 3|             12|
|   Stranger Things 2|             11|
|Squid Game: Season 1|             10|
|     Stranger Things|             10|
|Better Call Saul:...|             10|
|Bridgerton: Season 2|              9|
|Maid: Limited Series|              9|
|PAW Patrol: Season 6|              9|
|Inventing Anna: L...|              8|
| Money Heist: Part 5|              7|
|  Sex/Life: Season 1|              7|
|PIECES OF HER: Se...|              7|
|Clickbait: Limite...|              6|
|Sex Education: Se...|              6|
|Virgin River: Sea...|              6|
|Young Sheldon: : ...|              6|
|Young Sheldon: : ...|              6|
|The Lincoln Lawye...|              6|
|       You: Season 3|              6|
|PAW Patrol: Season 4|              6|
|The Umbrella Acad...|   

# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [59]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10

YSDF = (tvDF.filter('show_title == "Young Sheldon"')
        .groupBy('season_title', 'country_name')
        .agg(F.max('cumulative_weeks_in_top_10').alias('weeks_in_top_10'))
        .sort(F.desc('weeks_in_top_10')))

(YSDF.groupBy('season_title')
.agg(F.max('weeks_in_top_10').alias('weeks_in_top_10')) # taking only the max weeks in top 10
.join(YSDF, ['season_title', 'weeks_in_top_10']) # joining to get country_name column
.select('season_title', 'country_name', 'weeks_in_top_10') # reordering columns
.show())

+--------------------+--------------+---------------+
|        season_title|  country_name|weeks_in_top_10|
+--------------------+--------------+---------------+
|Young Sheldon: Se...|     Australia|              1|
|Young Sheldon: Se...|   New Zealand|              1|
|Young Sheldon: : ...|United Kingdom|              6|
|Young Sheldon: : ...|       Ireland|              6|
|Young Sheldon: : ...|United Kingdom|              6|
|Young Sheldon: : ...|        Canada|              6|
|Young Sheldon: : ...|United Kingdom|              4|
+--------------------+--------------+---------------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.

In [60]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.

films_weeks_in_top_10 = (filmsDF.groupBy('country_name', 'show_title')
                        .agg(F.max('cumulative_weeks_in_top_10')
                             .alias('weeks_in_top_10'))
                        .sort(F.desc('weeks_in_top_10')))

(films_weeks_in_top_10.groupBy('country_name')
.agg(F.max('weeks_in_top_10').alias('weeks_in_top_10')) # taking only the max weeks in top 10
.join(films_weeks_in_top_10, ['country_name', 'weeks_in_top_10']) # joining to get show_title column
.select('country_name', 'show_title', 'weeks_in_top_10') # reordering columns
.sort('country_name', 'show_title')
.show())

+--------------+--------------------+---------------+
|  country_name|          show_title|weeks_in_top_10|
+--------------+--------------------+---------------+
|     Argentina|  Sonic the Hedgehog|              7|
|     Australia| Back to the Outback|              8|
|       Austria|Harry Potter and ...|              9|
|       Bahamas|A Madea Family Fu...|              9|
|       Bahamas|  A Madea Homecoming|              9|
|       Bahrain|          Red Notice|              8|
|    Bangladesh|        Sooryavanshi|             21|
|       Belgium|    The Adam Project|              8|
|       Bolivia|               Shrek|             18|
|        Brazil|Hotel Transylvani...|              7|
|      Bulgaria|          Red Notice|             15|
|        Canada|How the Grinch St...|              7|
|        Canada|    The Adam Project|              7|
|         Chile|Hotel Transylvani...|              8|
|         Chile|  Sonic the Hedgehog|              8|
|      Colombia|          Re

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [61]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spens in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"

filmsDF = filmsDF.select(F.col('*'), F.when(F.col('weekly_rank') == 1, 1)
                .otherwise(0)
                .alias('weeks_at_1'))
filmsDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+----------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|weeks_at_1|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+----------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|         0|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|         0|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|         0|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|               

In [62]:
# CodeGrade Tag Ex6b

### Group by country name and sow title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.

films_weeks_at_1 = (filmsDF.groupBy('country_name', 'show_title')
                   .agg(F.sum('weeks_at_1').alias('weeks_at_1'))
                   .sort(F.desc('weeks_at_1')))

films_weeks_at_1.show(truncate=False)

+------------+-------------------+----------+
|country_name|show_title         |weeks_at_1|
+------------+-------------------+----------+
|Russia      |Don't Look Up      |6         |
|Hong Kong   |Don't Look Up      |6         |
|Taiwan      |Don't Look Up      |5         |
|Taiwan      |Gangubai Kathiawadi|5         |
|Vietnam     |Red Notice         |5         |
|Israel      |The Tinder Swindler|4         |
|Estonia     |Don't Look Up      |4         |
|Bangladesh  |Haseen Dillruba    |4         |
|Ukraine     |The Tinder Swindler|4         |
|Vietnam     |Don't Look Up      |4         |
|Croatia     |The Tinder Swindler|4         |
|Bulgaria    |Red Notice         |4         |
|India       |Haseen Dillruba    |4         |
|Jordan      |Red Notice         |4         |
|Russia      |The Tinder Swindler|4         |
|Slovenia    |Don't Look Up      |4         |
|Pakistan    |Haseen Dillruba    |4         |
|Denmark     |The Tinder Swindler|4         |
|Estonia     |The Tinder Swindler|

In [63]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.

(films_weeks_at_1
.groupBy('country_name')
.agg(F.max('weeks_at_1').alias('weeks_at_1')) # taking only the top film
.join(films_weeks_at_1, ['country_name', 'weeks_at_1']) # joining to get show_title column
.select('country_name', 'show_title', 'weeks_at_1') # selecting required column only
.sort('country_name', 'show_title')
.show())

+------------+--------------------+----------+
|country_name|          show_title|weeks_at_1|
+------------+--------------------+----------+
|   Argentina|       Don't Look Up|         3|
|   Australia|    The Adam Project|         3|
|     Austria| The Tinder Swindler|         4|
|     Bahamas|  A Madea Homecoming|         2|
|     Bahamas|             Bruised|         2|
|     Bahamas|           Day Shift|         2|
|     Bahamas|              Fatale|         2|
|     Bahamas|              Hustle|         2|
|     Bahamas|          Red Notice|         2|
|     Bahamas|      Resort to Love|         2|
|     Bahamas|         Senior Year|         2|
|     Bahamas|          Sweet Girl|         2|
|     Bahamas|        The Gray Man|         2|
|     Bahamas|The Man from Toronto|         2|
|     Bahamas|       The Sea Beast|         2|
|     Bahrain|       Don't Look Up|         3|
|     Bahrain| Gangubai Kathiawadi|         3|
|     Bahrain|          Red Notice|         3|
|     Bahrain